In [1]:
%load_ext autoreload
%autoreload 2

In [116]:
from utils.ds_utils import load_dataset_from_huggingface
ds = load_dataset_from_huggingface()

Generating train split: 100%|██████████| 45091/45091 [00:00<00:00, 86729.65 examples/s]

Loaded dataset: dragonslayer631/ci2_allsides, split: train


In [120]:
ds[0]

{'topic': 'economy_and_jobs',
 'tags': 'Economy And Jobs, Economic Policy, Budget Reconciliation, Infrastructure, Infrastructure Bill, Debt Ceiling',
 'text': 'We use cookies and similar methods to recognize visitors and remember their preferences. We may also use them to measure ad campaign effectiveness, target ads, and analyze site traffic. Depending on your location, you may opt-in or opt out of the use of these technologies. Search Illustration: Sarah Grillo/Axios The Senate\'s bipartisan $1.2 trillion infrastructure bill and President Biden\'s $3.5 trillion spending package could live or die this week — and take Democrats\' fortunes with them. But all the minute-by-minute political drama obscures how much America could change if even a fraction of it passes. The big picture: Anything short of total failure could have a transformative impact on day-to-day life — from how we move around to our access to the internet, paid family leave and child care, health care and college. Here\'

In [43]:
from datasets import Dataset
def update_dataset_column(ds: Dataset, values: dict, column: str):
    for i in range(len(ds)):
        if ds[i]['id'] in values.keys():
            ds[i][column] = values[ds[i]['id']]
    return ds

In [ ]:
from utils.groq_utils import generate_entites_sentiments_groq, get_groq_client
import json
from datasets import Dataset

client = get_groq_client()

def get_entity_sentiments_for_text_in_dataset(text: str, ds: Dataset):
    filtered = ds.filter(lambda x: x[f'{text}_entity_sentiments'] == None)
    print(f"Number of rows to process: {len(filtered)}")
    ctr = 0
    new_values = {}
    for elt in filtered:
        ctr += 1
        print(ctr)
        try:
            if text == 'text':
                # snip text
                text = elt['text'].split(' ')
                text = ' '.join(text[:1000])
            else:
                text = elt[text]
            sentiments = generate_entites_sentiments_groq(client=client, text=text, model="llama3-70b-8192")
            print("sentiments", sentiments)
            with open(f"data/{text}/{text}_sentiment_total.csv", "a") as f:
                f.write(f"{elt['id']}|{json.dumps(sentiments)}\n")
            new_values[elt['id']] = json.dumps(sentiments)
        except Exception as e:
            print("failed at", elt['id'], e)

    print(f"updating {text}_entity_sentiments with values ", new_values)
    update_dataset_column(ds, new_values, f'{text}_entity_sentiments')
    return ds
    



In [121]:
get_entity_sentiments_for_text_in_dataset("summary_50", ds)

Filter: 100%|██████████| 45091/45091 [00:01<00:00, 27233.55 examples/s]


Number of rows to process: 0
updating summary_50_entity_sentiments with values  {}


Dataset({
    features: ['topic', 'tags', 'text', 'int_bias', 'summary_5', 'summary_50', 'summary_100', 'text_entity_sentiments', 'text_topic_to_sentiment', 'summary_100_entity_sentiments', 'summary_100_topic_to_sentiment', 'summary_50_entity_sentiments', 'summary_50_topic_to_sentiment', 'id'],
    num_rows: 45091
})